In [52]:
import numpy as np
from typing import Self
from copy import deepcopy

In [53]:
class SudokuBoard:
    def __init__(self):
        self._board = np.matrix(np.zeros((9,9),np.int64))
        
    def __str__(self):
        rows = []
        rows.append('#+------+-------+------+')
        for i in range(9):
            row = []
            for e in self._board[i, :].A1: 
                if e == 0:
                    row.append(' ')
                else:
                    row.append(str(e))
                    
            row.insert(3, '|')
            row.insert(7, '|')
            rows.append('#|'+' '.join(row)+'|')

            if i == 2 or i == 5 or i == 8:
                rows.append('#+------+-------+------+')

        return '\n'.join(rows)
    
    def at(self, row, col):
        return self._board[row, col]
    
    def as_matrix(self):
        return self._board
    
    def set(self, row, col, value):
        if row >= 0 and row < 9 and col >= 0 and col < 9:
            self._board[row, col] = value
            return True
        return False
    
    def set_all(self,values):
        for value in values:
            self.set(value[0], value[1], value[2])
    
    def is_solved(self):        
        rows_ok, cols_ok, squares_ok = self.solution_status()
        return rows_ok.all() and cols_ok.all() and squares_ok.all()
    
    def solution_status(self):
        rows_ok = np.apply_along_axis(
            SudokuBoard.check_1D, 
            axis=1, 
            arr=np.asarray(self._board)
        ).flatten()
        
        cols_ok = np.apply_along_axis(
            SudokuBoard.check_1D,
            axis=0,
            arr=np.asarray(self._board) 
        ).flatten()
        
        squares_ok = SudokuBoard.check_squares(self._board)
        
        return rows_ok, cols_ok, squares_ok
    
    @staticmethod
    def check_1D(row_or_col):
        numbers = set(list(range(1, 10)))
        return set(row_or_col) == numbers
    
    @staticmethod
    def check_squares(board):
        numbers = set(list(range(1, 10)))
        squares_ok = np.empty(9)
        for square in range(9):
            rows = range(3 * (1 + int(square/3)))
            cols = range(3 * (1 + square % 3))
            sub_board = board[rows, :][:, cols]
            squares_ok[square] = set(sub_board.A1) == numbers
        return squares_ok.astype(bool)

In [132]:
class SudokuSolver:
    def __init__(self, T_initial = 200, T_min = -np.inf, alpha = 1.1, max_steps=10000):
        self.steps = 0
        self.T = T_initial
        self.T_min = T_min
        self.alpha = alpha
        self.max_steps = max_steps
        self.solution = None
        self.original_board = None
        
    def print(self):
        print(self.solution)
    
    def solve(self, board: SudokuBoard) -> Self:
        self.original_board = deepcopy(board)
        self.solution = deepcopy(board)
        if self.solution.is_solved():
            return self
        while True:
            neighbors = self._board_neighbors(self.solution)
            for neighbor in neighbors:
                energy_variation = SudokuSolver.board_energy(neighbor) - SudokuSolver.board_energy(self.solution)
                if energy_variation < 0:
                    self.solution = deepcopy(neighbor)
                    self.steps += 1
                else:
                    p = np.exp(-energy_variation/self.T)
                    n = np.random.uniform(0, 1)
                    if n < p:
                        self.solution = deepcopy(neighbor)
                        self.steps += 1       
                self._temperature_schedule() 
                if self.T < self.T_min or self.solution.is_solved() or self.steps > self.max_steps:
                    print(f"Step: {self.steps}")
                    print(f"Energy: {SudokuSolver.board_energy(self.solution)}")
                    print(f"Temperature: {self.T}")
                    return self
                if self.steps == 1 or self.steps % 50 == 0:
                    print(f"Step: {self.steps}")
                    print(f"Energy variation: {energy_variation}")
                    print(f"Energy: {SudokuSolver.board_energy(self.solution)}")
                    print(f"Temperature: {self.T}")

    def _board_neighbors(self, board: SudokuBoard):
        numbers = list(range(9))
        neighbors = []
        for row in numbers:
            for col in numbers:
                if self.original_board.at(row, col) == 0:
                    for number in numbers + [9]:
                        if number != 0:
                            neighbor = deepcopy(board)
                            neighbor.set(row, col, number)
                            neighbors.append(neighbor)
        return neighbors
    
    def _temperature_schedule(self):
        self.T = self.T / self.alpha
        
    @staticmethod
    def board_energy(board: SudokuBoard):
        matrix = board.as_matrix()
        
        rows_losses = np.apply_along_axis(
            SudokuSolver.row_or_col_loss,
            axis=0,
            arr=np.asarray(matrix) 
        ).flatten()
        
        cols_losses = np.apply_along_axis(
            SudokuSolver.row_or_col_loss,
            axis=1,
            arr=np.asarray(matrix) 
        ).flatten()
        
        squares_loss = SudokuSolver.squares_loss(matrix)
        completeness_loss = rows_losses.sum() + cols_losses.sum() + squares_loss
        
        rows_ok, cols_ok, squares_ok = board.solution_status()
        rule_loss = np.logical_not(np.concat((rows_ok, cols_ok, squares_ok))).sum()
        
        return completeness_loss + 20 * rule_loss
    
    @staticmethod
    def row_or_col_loss(row_or_col):
        numbers = set(list(range(1, 10)))
        return len(numbers.difference(set(list(row_or_col))))

    @staticmethod
    def squares_loss(matrix):
        numbers = set(list(range(1, 10)))
        squares_losses = np.empty(9)
        for square in range(9):
            rows = range(3 * (1 + int(square/3)))
            cols = range(3 * (1 + square % 3))
            sub_matrix = matrix[rows, :][:, cols]
            squares_losses[square] = len(numbers.difference(set(list(sub_matrix.A1))))
        return squares_losses.sum()

In [133]:
board = SudokuBoard()

board.set_all([
    [0,0,2],[0,1,5],[0,5,3],[0,7,9],[0,8,1],
    [1,1,1]
])

board.set(1,5,4)
print(board)

#+------+-------+------+
#|2 5   |     3 |   9 1|
#|  1   |     4 |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+


In [134]:
agent = SudokuSolver(T_initial=10000, T_min=-np.inf, alpha=1.001, max_steps=15000)
agent = agent.solve(board)

Step: 1
Energy variation: -1.0
Energy: 726.0
Temperature: 9990.00999000999
Step: 50
Energy variation: 2.0
Energy: 725.0
Temperature: 9512.531896912551
Step: 100
Energy variation: 0.0
Energy: 725.0
Temperature: 9048.826308977857
Step: 150
Energy variation: -6.0
Energy: 719.0
Temperature: 8607.724889377332
Step: 200
Energy variation: 0.0
Energy: 725.0
Temperature: 8188.125757004989
Step: 250
Energy variation: 0.0
Energy: 723.0
Temperature: 7788.9807439441165
Step: 300
Energy variation: 0.0
Energy: 725.0
Temperature: 7409.2927771206005
Step: 350
Energy variation: 0.0
Energy: 719.0
Temperature: 7048.113387592345
Step: 400
Energy variation: 0.0
Energy: 725.0
Temperature: 6704.540341252855
Step: 450
Energy variation: 1.0
Energy: 723.0
Temperature: 6377.715385030471
Step: 500
Energy variation: 0.0
Energy: 725.0
Temperature: 6066.822102953223
Step: 550
Energy variation: 1.0
Energy: 725.0
Temperature: 5771.083876723657
Step: 600
Energy variation: -4.0
Energy: 722.0
Temperature: 5489.76194570914

In [136]:
agent.print()

#+------+-------+------+
#|2 5 4 |     3 |   9 1|
#|3 1 9 |     4 |      |
#|7 8 6 |       |      |
#+------+-------+------+
#|      |       |   2  |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       | 8 6 7|
#|1 4 2 | 3 6 5 | 7 8 9|
#|1 2 3 | 4 5 6 | 9 7 8|
#+------+-------+------+
